<a href="https://colab.research.google.com/github/CarolineAndradeR/Python/blob/master/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Criando a sessão Spark
spark = SparkSession.builder.appName("ExerciciosFuncionarios").getOrCreate()

# Definindo o schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("nome", StringType(), True),
    StructField("departamento", StringType(), True),
    StructField("cargo", StringType(), True),
    StructField("idade", IntegerType(), True),
    StructField("salario", DoubleType(), True),
    StructField("cidade", StringType(), True)
])

# Inserindo dados fictícios
dados = [
    (1, "Ana Souza", "RH", "Analista de RH", 29, 3200.0, "São Paulo"),
    (2, "Carlos Lima", "TI", "Desenvolvedor", 34, 7200.0, "Rio de Janeiro"),
    (3, "Fernanda Alves", "Financeiro", "Contadora", 41, 6500.0, "Curitiba"),
    (4, "Ricardo Silva", "Vendas", "Vendedor", 28, 2800.0, "Belo Horizonte"),
    (5, "Mariana Costa", "Marketing", "Coordenadora de Marketing", 37, 7200.0, "São Paulo"),
    (6, "João Pedro", "Logística", "Auxiliar de Logística", 23, 2100.0, "Recife"),
    (7, "Patrícia Gomes", "Operações", "Gerente de Operações", 46, 11000.0, "Fortaleza"),
    (8, "Gabriel Ferreira", "TI", "Cientista de Dados", 31, 12500.0, "Porto Alegre"),
    (9, "Larissa Martins", "Financeiro", "Analista Financeiro", 26, 3900.0, "Rio de Janeiro"),
    (10, "Bruno Rocha", "Vendas", "Supervisor de Vendas", 39, 5400.0, "Campinas"),
    (11, "Camila Torres", "Marketing", "Assistente de Marketing", 24, 2500.0, "São Paulo"),
    (12, "Eduardo Ramos", "TI", "Engenheiro de Software", 36, 9800.0, "Brasília"),
    (13, "Juliana Melo", "RH", "Coordenadora de RH", 42, 8700.0, "Salvador"),
    (14, "André Santos", "Financeiro", "Diretor Financeiro", 50, 15000.0, "São Paulo"),
    (15, "Roberta Araújo", "Operações", "Analista de Operações", 30, 4300.0, "Recife"),
    (16, "Felipe Cardoso", "TI", "Analista de Suporte", 27, 3100.0, "Curitiba"),
    (17, "Beatriz Moura", "Vendas", "Executiva de Contas", 33, 6100.0, "Rio de Janeiro"),
    (18, "Marcelo Pinto", "Logística", "Motorista", 45, 2800.0, "Manaus"),
    (19, "Tatiane Lima", "Marketing", "Gerente de Marketing", 38, 9500.0, "São Paulo"),
    (20, "Diego Nunes", "TI", "Arquiteto de Software", 40, 13200.0, "Porto Alegre"),
    (21, "Amanda Silva", "RH", "Recrutadora", 27, 3100.0, "Rio de Janeiro"),
    (22, "Leonardo Alves", "Financeiro", "Tesoureiro", 48, 8700.0, "São Paulo"),
    (23, "Paula Martins", "TI", "Desenvolvedora Frontend", 29, 7100.0, "Fortaleza"),
    (24, "Thiago Lima", "Vendas", "Representante Comercial", 35, 4800.0, "Curitiba"),
    (25, "Isabela Costa", "Marketing", "Analista de Comunicação", 26, 3300.0, "Brasília"),
    (26, "Rodrigo Ferreira", "Operações", "Supervisor de Produção", 44, 8200.0, "São Paulo"),
    (27, "Marcos Vinicius", "Logística", "Analista de Transporte", 31, 4100.0, "Recife"),
    (28, "Sabrina Souza", "RH", "Assistente de RH", 24, 2400.0, "Belo Horizonte"),
    (29, "Joana Pereira", "Financeiro", "Analista Contábil", 39, 5900.0, "Porto Alegre"),
    (30, "Lucas Rocha", "TI", "Desenvolvedor Backend", 32, 7800.0, "São Paulo"),
    (31, "Natália Dias", "Vendas", "Consultora de Vendas", 28, 3500.0, "Curitiba"),
    (32, "Anderson Gomes", "Operações", "Operador de Máquinas", 40, 2700.0, "Manaus"),
    (33, "Gabriela Torres", "Marketing", "Social Media", 25, 2900.0, "Rio de Janeiro"),
    (34, "Rafael Santos", "TI", "DevOps Engineer", 37, 10500.0, "Brasília"),
    (35, "Luciana Almeida", "Financeiro", "Coordenadora Financeira", 45, 9400.0, "São Paulo"),
    (36, "Pedro Henrique", "RH", "Business Partner", 41, 8700.0, "Fortaleza"),
    (37, "Cíntia Barbosa", "Logística", "Coordenadora de Logística", 38, 7200.0, "Salvador"),
    (38, "Bruna Cardoso", "Marketing", "Designer Gráfico", 29, 4100.0, "Recife"),
    (39, "Mateus Lima", "Vendas", "Gerente de Vendas", 43, 11800.0, "São Paulo"),
    (40, "Débora Nogueira", "TI", "QA Engineer", 30, 6600.0, "Rio de Janeiro"),
    (41, "Rodrigo Almeida", "Operações", "Diretor de Operações", 50, 14200.0, "Brasília"),
    (42, "Vanessa Mendes", "Financeiro", "Assistente Financeiro", 23, 2600.0, "Curitiba"),
    (43, "Cláudio Ribeiro", "Logística", "Supervisor de Armazém", 46, 6400.0, "São Paulo"),
    (44, "Carolina Freitas", "RH", "Coordenadora de Treinamento", 34, 5200.0, "Fortaleza"),
    (45, "Gustavo Oliveira", "TI", "Estagiário de TI", 21, 1800.0, "Recife"),
    (46, "Patrícia Nunes", "Marketing", "Gerente de Produto", 37, 9700.0, "Rio de Janeiro"),
    (47, "Felipe Fernandes", "Financeiro", "Auditor", 42, 8900.0, "São Paulo"),
    (48, "Juliana Castro", "Operações", "Supervisora de Operações", 36, 7500.0, "Manaus"),
    (49, "Alexandre Lima", "Logística", "Analista de Estoque", 27, 3300.0, "Belo Horizonte"),
    (50, "Camila Oliveira", "Vendas", "Executiva de Negócios", 31, 6700.0, "Curitiba"),
    (51, "Daniel Pereira", "TI", "Especialista em Redes", 39, 9100.0, "São Paulo"),
    (52, "Larissa Barbosa", "Financeiro", "Consultora Financeira", 35, 7200.0, "Fortaleza"),
    (53, "Roberto Souza", "RH", "Gerente de RH", 47, 11200.0, "Recife"),
    (54, "Fernanda Rocha", "Marketing", "Analista de Marketing", 28, 3500.0, "Brasília"),
    (55, "João Marcos", "Logística", "Motorista", 44, 2900.0, "Salvador"),
    (56, "Tatiane Ferreira", "Operações", "Coordenadora de Operações", 40, 8200.0, "São Paulo"),
    (57, "Carlos Eduardo", "TI", "Programador Mobile", 29, 7400.0, "Porto Alegre"),
    (58, "Mariana Ribeiro", "Financeiro", "Diretora Financeira", 49, 14800.0, "São Paulo"),
    (59, "Renato Gomes", "Vendas", "Supervisor de Vendas", 37, 5600.0, "Fortaleza"),
    (60, "Amanda Torres", "Marketing", "Assistente de Comunicação", 22, 2300.0, "Rio de Janeiro")
]

# Criando o DataFrame
df = spark.createDataFrame(data=dados, schema=schema)

# Registrando como tabela temporária para SQL
df.createOrReplaceTempView("funcionarios")

# Visualizando os dados
df.show(truncate=False)


+---+----------------+------------+-------------------------+-----+-------+--------------+
|id |nome            |departamento|cargo                    |idade|salario|cidade        |
+---+----------------+------------+-------------------------+-----+-------+--------------+
|1  |Ana Souza       |RH          |Analista de RH           |29   |3200.0 |São Paulo     |
|2  |Carlos Lima     |TI          |Desenvolvedor            |34   |7200.0 |Rio de Janeiro|
|3  |Fernanda Alves  |Financeiro  |Contadora                |41   |6500.0 |Curitiba      |
|4  |Ricardo Silva   |Vendas      |Vendedor                 |28   |2800.0 |Belo Horizonte|
|5  |Mariana Costa   |Marketing   |Coordenadora de Marketing|37   |7200.0 |São Paulo     |
|6  |João Pedro      |Logística   |Auxiliar de Logística    |23   |2100.0 |Recife        |
|7  |Patrícia Gomes  |Operações   |Gerente de Operações     |46   |11000.0|Fortaleza     |
|8  |Gabriel Ferreira|TI          |Cientista de Dados       |31   |12500.0|Porto Alegre  |

# **Filtros e Seleções**

In [52]:
# Seleção de colunas especificas
df.select("nome", "salario").show()

+----------------+-------+
|            nome|salario|
+----------------+-------+
|       Ana Souza| 3200.0|
|     Carlos Lima| 7200.0|
|  Fernanda Alves| 6500.0|
|   Ricardo Silva| 2800.0|
|   Mariana Costa| 7200.0|
|      João Pedro| 2100.0|
|  Patrícia Gomes|11000.0|
|Gabriel Ferreira|12500.0|
| Larissa Martins| 3900.0|
|     Bruno Rocha| 5400.0|
|   Camila Torres| 2500.0|
|   Eduardo Ramos| 9800.0|
|    Juliana Melo| 8700.0|
|    André Santos|15000.0|
|  Roberta Araújo| 4300.0|
|  Felipe Cardoso| 3100.0|
|   Beatriz Moura| 6100.0|
|   Marcelo Pinto| 2800.0|
|    Tatiane Lima| 9500.0|
|     Diego Nunes|13200.0|
+----------------+-------+
only showing top 20 rows



In [53]:
# Filtro (where)
df.filter(df.salario > 2500).show()

+---+----------------+------------+--------------------+-----+-------+--------------+
| id|            nome|departamento|               cargo|idade|salario|        cidade|
+---+----------------+------------+--------------------+-----+-------+--------------+
|  1|       Ana Souza|          RH|      Analista de RH|   29| 3200.0|     São Paulo|
|  2|     Carlos Lima|          TI|       Desenvolvedor|   34| 7200.0|Rio de Janeiro|
|  3|  Fernanda Alves|  Financeiro|           Contadora|   41| 6500.0|      Curitiba|
|  4|   Ricardo Silva|      Vendas|            Vendedor|   28| 2800.0|Belo Horizonte|
|  5|   Mariana Costa|   Marketing|Coordenadora de M...|   37| 7200.0|     São Paulo|
|  7|  Patrícia Gomes|   Operações|Gerente de Operações|   46|11000.0|     Fortaleza|
|  8|Gabriel Ferreira|          TI|  Cientista de Dados|   31|12500.0|  Porto Alegre|
|  9| Larissa Martins|  Financeiro| Analista Financeiro|   26| 3900.0|Rio de Janeiro|
| 10|     Bruno Rocha|      Vendas|Supervisor de Venda

In [54]:
#Agrupamento por coluna
df.groupBy("departamento").avg("salario").show()

+------------+-----------------+
|departamento|     avg(salario)|
+------------+-----------------+
|          TI|8008.333333333333|
|      Vendas|           5837.5|
|   Logística|4114.285714285715|
|          RH|6071.428571428572|
|  Financeiro|           8290.0|
|   Marketing|           5000.0|
|   Operações|8014.285714285715|
+------------+-----------------+



# **Spark SQL**

In [55]:
# Antes de rodar o SQL é necessário criar um Dataframe como tabela temporária
df.createOrReplaceTempView("funcionarios")

In [56]:
spark.sql("SELECT nome, salario FROM funcionarios").show()

+----------------+-------+
|            nome|salario|
+----------------+-------+
|       Ana Souza| 3200.0|
|     Carlos Lima| 7200.0|
|  Fernanda Alves| 6500.0|
|   Ricardo Silva| 2800.0|
|   Mariana Costa| 7200.0|
|      João Pedro| 2100.0|
|  Patrícia Gomes|11000.0|
|Gabriel Ferreira|12500.0|
| Larissa Martins| 3900.0|
|     Bruno Rocha| 5400.0|
|   Camila Torres| 2500.0|
|   Eduardo Ramos| 9800.0|
|    Juliana Melo| 8700.0|
|    André Santos|15000.0|
|  Roberta Araújo| 4300.0|
|  Felipe Cardoso| 3100.0|
|   Beatriz Moura| 6100.0|
|   Marcelo Pinto| 2800.0|
|    Tatiane Lima| 9500.0|
|     Diego Nunes|13200.0|
+----------------+-------+
only showing top 20 rows



In [57]:
spark.sql("SELECT * FROM funcionarios WHERE salario > 2500").show()

+---+----------------+------------+--------------------+-----+-------+--------------+
| id|            nome|departamento|               cargo|idade|salario|        cidade|
+---+----------------+------------+--------------------+-----+-------+--------------+
|  1|       Ana Souza|          RH|      Analista de RH|   29| 3200.0|     São Paulo|
|  2|     Carlos Lima|          TI|       Desenvolvedor|   34| 7200.0|Rio de Janeiro|
|  3|  Fernanda Alves|  Financeiro|           Contadora|   41| 6500.0|      Curitiba|
|  4|   Ricardo Silva|      Vendas|            Vendedor|   28| 2800.0|Belo Horizonte|
|  5|   Mariana Costa|   Marketing|Coordenadora de M...|   37| 7200.0|     São Paulo|
|  7|  Patrícia Gomes|   Operações|Gerente de Operações|   46|11000.0|     Fortaleza|
|  8|Gabriel Ferreira|          TI|  Cientista de Dados|   31|12500.0|  Porto Alegre|
|  9| Larissa Martins|  Financeiro| Analista Financeiro|   26| 3900.0|Rio de Janeiro|
| 10|     Bruno Rocha|      Vendas|Supervisor de Venda

In [58]:
spark.sql("""
    SELECT departamento, AVG(salario) as media_salario
    FROM funcionarios
    GROUP BY departamento
""").show()

+------------+-----------------+
|departamento|    media_salario|
+------------+-----------------+
|          TI|8008.333333333333|
|      Vendas|           5837.5|
|   Logística|4114.285714285715|
|          RH|6071.428571428572|
|  Financeiro|           8290.0|
|   Marketing|           5000.0|
|   Operações|8014.285714285715|
+------------+-----------------+



# **Exercícios**

In [59]:
#Quais funcionários ganham menos de 3000?
df.filter(df.salario < 3000).show()

+---+----------------+------------+--------------------+-----+-------+--------------+
| id|            nome|departamento|               cargo|idade|salario|        cidade|
+---+----------------+------------+--------------------+-----+-------+--------------+
|  4|   Ricardo Silva|      Vendas|            Vendedor|   28| 2800.0|Belo Horizonte|
|  6|      João Pedro|   Logística|Auxiliar de Logís...|   23| 2100.0|        Recife|
| 11|   Camila Torres|   Marketing|Assistente de Mar...|   24| 2500.0|     São Paulo|
| 18|   Marcelo Pinto|   Logística|           Motorista|   45| 2800.0|        Manaus|
| 28|   Sabrina Souza|          RH|    Assistente de RH|   24| 2400.0|Belo Horizonte|
| 32|  Anderson Gomes|   Operações|Operador de Máquinas|   40| 2700.0|        Manaus|
| 33| Gabriela Torres|   Marketing|        Social Media|   25| 2900.0|Rio de Janeiro|
| 42|  Vanessa Mendes|  Financeiro|Assistente Financ...|   23| 2600.0|      Curitiba|
| 45|Gustavo Oliveira|          TI|    Estagiário de T

In [60]:
#Quais funcionários ganham menos de 3000?
spark.sql("SELECT * FROM funcionarios WHERE salario < 3000").show()

+---+----------------+------------+--------------------+-----+-------+--------------+
| id|            nome|departamento|               cargo|idade|salario|        cidade|
+---+----------------+------------+--------------------+-----+-------+--------------+
|  4|   Ricardo Silva|      Vendas|            Vendedor|   28| 2800.0|Belo Horizonte|
|  6|      João Pedro|   Logística|Auxiliar de Logís...|   23| 2100.0|        Recife|
| 11|   Camila Torres|   Marketing|Assistente de Mar...|   24| 2500.0|     São Paulo|
| 18|   Marcelo Pinto|   Logística|           Motorista|   45| 2800.0|        Manaus|
| 28|   Sabrina Souza|          RH|    Assistente de RH|   24| 2400.0|Belo Horizonte|
| 32|  Anderson Gomes|   Operações|Operador de Máquinas|   40| 2700.0|        Manaus|
| 33| Gabriela Torres|   Marketing|        Social Media|   25| 2900.0|Rio de Janeiro|
| 42|  Vanessa Mendes|  Financeiro|Assistente Financ...|   23| 2600.0|      Curitiba|
| 45|Gustavo Oliveira|          TI|    Estagiário de T

In [61]:
#Qual é o maior salário do departamento de RH?
from pyspark.sql.functions import max

df.filter(df.departamento=="RH").agg(max(df.salario)).show()

+------------+
|max(salario)|
+------------+
|     11200.0|
+------------+



In [62]:
#Qual é o maior salário do departamento de RH?
spark.sql("""
    SELECT max(salario) as maior_salario
    FROM funcionarios
    WHERE departamento = 'RH'
""").show()

+-------------+
|maior_salario|
+-------------+
|      11200.0|
+-------------+



In [63]:
#Liste todos os nomes em ordem decrescente de salário.
df.orderBy(df.salario.desc()).select("nome").show()

+----------------+
|            nome|
+----------------+
|    André Santos|
| Mariana Ribeiro|
| Rodrigo Almeida|
|     Diego Nunes|
|Gabriel Ferreira|
|     Mateus Lima|
|   Roberto Souza|
|  Patrícia Gomes|
|   Rafael Santos|
|   Eduardo Ramos|
|  Patrícia Nunes|
|    Tatiane Lima|
| Luciana Almeida|
|  Daniel Pereira|
|Felipe Fernandes|
|    Juliana Melo|
|  Pedro Henrique|
|  Leonardo Alves|
|Tatiane Ferreira|
|Rodrigo Ferreira|
+----------------+
only showing top 20 rows



In [64]:
#Liste todos os nomes em ordem decrescente de salário.
spark.sql("""
    SELECT nome
    FROM funcionarios
    ORDER BY salario DESC
""").show()

+----------------+
|            nome|
+----------------+
|    André Santos|
| Mariana Ribeiro|
| Rodrigo Almeida|
|     Diego Nunes|
|Gabriel Ferreira|
|     Mateus Lima|
|   Roberto Souza|
|  Patrícia Gomes|
|   Rafael Santos|
|   Eduardo Ramos|
|  Patrícia Nunes|
|    Tatiane Lima|
| Luciana Almeida|
|  Daniel Pereira|
|Felipe Fernandes|
|    Juliana Melo|
|  Pedro Henrique|
|  Leonardo Alves|
|Tatiane Ferreira|
|Rodrigo Ferreira|
+----------------+
only showing top 20 rows



In [65]:
#Qual o salário médio geral da empresa?
from pyspark.sql import functions as F

df.agg(F.avg("salario").alias("salario_medio")).show()

+-------------+
|salario_medio|
+-------------+
|       6635.0|
+-------------+



In [66]:
#Qual o salário médio geral da empresa?
spark.sql("""
    SELECT AVG(salario) as salario_medio
    FROM funcionarios
    """).show()

+-------------+
|salario_medio|
+-------------+
|       6635.0|
+-------------+



In [67]:
# Salário médio por departamento
df.groupBy("departamento").agg(F.avg("salario").alias("salario_medio")).show()

+------------+-----------------+
|departamento|    salario_medio|
+------------+-----------------+
|          TI|8008.333333333333|
|      Vendas|           5837.5|
|   Logística|4114.285714285715|
|          RH|6071.428571428572|
|  Financeiro|           8290.0|
|   Marketing|           5000.0|
|   Operações|8014.285714285715|
+------------+-----------------+



In [68]:
# Salário médio por departamento
spark.sql("""
    SELECT departamento, AVG(salario) as salario_medio
    FROM funcionarios
    GROUP BY departamento
""").show()

+------------+-----------------+
|departamento|    salario_medio|
+------------+-----------------+
|          TI|8008.333333333333|
|      Vendas|           5837.5|
|   Logística|4114.285714285715|
|          RH|6071.428571428572|
|  Financeiro|           8290.0|
|   Marketing|           5000.0|
|   Operações|8014.285714285715|
+------------+-----------------+



In [69]:
# Maior e menor salário da empresa
from pyspark.sql.functions import min
df.agg(min("salario"),max("salario")).show()

+------------+------------+
|min(salario)|max(salario)|
+------------+------------+
|      1800.0|     15000.0|
+------------+------------+



In [70]:
#Maior e menor salário da empresa
spark.sql("""
    select min(salario) as menor, max(salario) as maior
    from funcionarios
""").show()

+------+-------+
| menor|  maior|
+------+-------+
|1800.0|15000.0|
+------+-------+



In [71]:
# Quantos funcionários existem em cada cargo?
from pyspark.sql.functions import count
df.groupBy("departamento").agg(count("nome").alias("quantidade_funcionarios")).show()

+------------+-----------------------+
|departamento|quantidade_funcionarios|
+------------+-----------------------+
|          TI|                     12|
|      Vendas|                      8|
|   Logística|                      7|
|          RH|                      7|
|  Financeiro|                     10|
|   Marketing|                      9|
|   Operações|                      7|
+------------+-----------------------+



In [72]:
# Quantos funcionários existem em cada cargo?
spark.sql("""
    select departamento, count(nome) as quantidade
    from funcionarios
    group by 1
""").show()

+------------+----------+
|departamento|quantidade|
+------------+----------+
|          TI|        12|
|      Vendas|         8|
|   Logística|         7|
|          RH|         7|
|  Financeiro|        10|
|   Marketing|         9|
|   Operações|         7|
+------------+----------+



In [73]:
#Soma dos salários de todos os funcionários de TI
df.filter(df.departamento=="TI").agg(F.sum(df.salario).alias("salarios_ti")).show()

+-----------+
|salarios_ti|
+-----------+
|    96100.0|
+-----------+



In [74]:
#Soma dos salários de todos os funcionários de TI
spark.sql("""
    select sum(salario) as salarios_ti
    from funcionarios
    where departamento = "TI"
""").show()

+-----------+
|salarios_ti|
+-----------+
|    96100.0|
+-----------+



In [75]:
# Média salarial considerando apenas salários acima de 5.000
df.filter(df.salario>5000).agg(F.avg(df.salario).alias("media_salarial")).show()

+-----------------+
|   media_salarial|
+-----------------+
|8813.513513513513|
+-----------------+



# Exercícios Básicos

In [79]:
# Liste todos os funcionários do departamento TI.
df.filter(df.departamento=="TI").show()

+---+----------------+------------+--------------------+-----+-------+--------------+
| id|            nome|departamento|               cargo|idade|salario|        cidade|
+---+----------------+------------+--------------------+-----+-------+--------------+
|  2|     Carlos Lima|          TI|       Desenvolvedor|   34| 7200.0|Rio de Janeiro|
|  8|Gabriel Ferreira|          TI|  Cientista de Dados|   31|12500.0|  Porto Alegre|
| 12|   Eduardo Ramos|          TI|Engenheiro de Sof...|   36| 9800.0|      Brasília|
| 16|  Felipe Cardoso|          TI| Analista de Suporte|   27| 3100.0|      Curitiba|
| 20|     Diego Nunes|          TI|Arquiteto de Soft...|   40|13200.0|  Porto Alegre|
| 23|   Paula Martins|          TI|Desenvolvedora Fr...|   29| 7100.0|     Fortaleza|
| 30|     Lucas Rocha|          TI|Desenvolvedor Bac...|   32| 7800.0|     São Paulo|
| 34|   Rafael Santos|          TI|     DevOps Engineer|   37|10500.0|      Brasília|
| 40| Débora Nogueira|          TI|         QA Enginee

In [81]:
# Liste todos os funcionários do departamento TI.
spark.sql("""
    select *
    from funcionarios
    where departamento = "TI"
""").show()

+---+----------------+------------+--------------------+-----+-------+--------------+
| id|            nome|departamento|               cargo|idade|salario|        cidade|
+---+----------------+------------+--------------------+-----+-------+--------------+
|  2|     Carlos Lima|          TI|       Desenvolvedor|   34| 7200.0|Rio de Janeiro|
|  8|Gabriel Ferreira|          TI|  Cientista de Dados|   31|12500.0|  Porto Alegre|
| 12|   Eduardo Ramos|          TI|Engenheiro de Sof...|   36| 9800.0|      Brasília|
| 16|  Felipe Cardoso|          TI| Analista de Suporte|   27| 3100.0|      Curitiba|
| 20|     Diego Nunes|          TI|Arquiteto de Soft...|   40|13200.0|  Porto Alegre|
| 23|   Paula Martins|          TI|Desenvolvedora Fr...|   29| 7100.0|     Fortaleza|
| 30|     Lucas Rocha|          TI|Desenvolvedor Bac...|   32| 7800.0|     São Paulo|
| 34|   Rafael Santos|          TI|     DevOps Engineer|   37|10500.0|      Brasília|
| 40| Débora Nogueira|          TI|         QA Enginee

In [95]:
# Liste apenas o nome e o salário dos funcionários que ganham mais de 5000.
df.filter((df.salario >= 5000) & (df.departamento == "TI")).select("nome", "salario", "departamento").show()

+----------------+-------+------------+
|            nome|salario|departamento|
+----------------+-------+------------+
|     Carlos Lima| 7200.0|          TI|
|Gabriel Ferreira|12500.0|          TI|
|   Eduardo Ramos| 9800.0|          TI|
|     Diego Nunes|13200.0|          TI|
|   Paula Martins| 7100.0|          TI|
|     Lucas Rocha| 7800.0|          TI|
|   Rafael Santos|10500.0|          TI|
| Débora Nogueira| 6600.0|          TI|
|  Daniel Pereira| 9100.0|          TI|
|  Carlos Eduardo| 7400.0|          TI|
+----------------+-------+------------+



In [96]:
# Liste apenas o nome e o salário dos funcionários que ganham mais de 5000 OU é do departamento TI.
df.filter((df.salario >= 5000) | (df.departamento == "TI")).select("nome", "salario", "departamento").show()

+----------------+-------+------------+
|            nome|salario|departamento|
+----------------+-------+------------+
|     Carlos Lima| 7200.0|          TI|
|  Fernanda Alves| 6500.0|  Financeiro|
|   Mariana Costa| 7200.0|   Marketing|
|  Patrícia Gomes|11000.0|   Operações|
|Gabriel Ferreira|12500.0|          TI|
|     Bruno Rocha| 5400.0|      Vendas|
|   Eduardo Ramos| 9800.0|          TI|
|    Juliana Melo| 8700.0|          RH|
|    André Santos|15000.0|  Financeiro|
|  Felipe Cardoso| 3100.0|          TI|
|   Beatriz Moura| 6100.0|      Vendas|
|    Tatiane Lima| 9500.0|   Marketing|
|     Diego Nunes|13200.0|          TI|
|  Leonardo Alves| 8700.0|  Financeiro|
|   Paula Martins| 7100.0|          TI|
|Rodrigo Ferreira| 8200.0|   Operações|
|   Joana Pereira| 5900.0|  Financeiro|
|     Lucas Rocha| 7800.0|          TI|
|   Rafael Santos|10500.0|          TI|
| Luciana Almeida| 9400.0|  Financeiro|
+----------------+-------+------------+
only showing top 20 rows



In [98]:
# Liste apenas o nome e o salário dos funcionários que ganham mais de 5000
df.filter(df.salario >= 5000).select("nome", "salario").orderBy("salario").show()

+----------------+-------+
|            nome|salario|
+----------------+-------+
|Carolina Freitas| 5200.0|
|     Bruno Rocha| 5400.0|
|    Renato Gomes| 5600.0|
|   Joana Pereira| 5900.0|
|   Beatriz Moura| 6100.0|
| Cláudio Ribeiro| 6400.0|
|  Fernanda Alves| 6500.0|
| Débora Nogueira| 6600.0|
| Camila Oliveira| 6700.0|
|   Paula Martins| 7100.0|
|  Cíntia Barbosa| 7200.0|
|     Carlos Lima| 7200.0|
| Larissa Barbosa| 7200.0|
|   Mariana Costa| 7200.0|
|  Carlos Eduardo| 7400.0|
|  Juliana Castro| 7500.0|
|     Lucas Rocha| 7800.0|
|Tatiane Ferreira| 8200.0|
|Rodrigo Ferreira| 8200.0|
|  Pedro Henrique| 8700.0|
+----------------+-------+
only showing top 20 rows



In [99]:
# Liste apenas o nome e o salário dos funcionários que ganham mais de 5000
spark.sql("""
    select nome, salario
    from funcionarios
    where salario >= 5000
""").show()

+----------------+-------+
|            nome|salario|
+----------------+-------+
|    André Santos|15000.0|
|   Beatriz Moura| 6100.0|
|     Bruno Rocha| 5400.0|
| Camila Oliveira| 6700.0|
|  Carlos Eduardo| 7400.0|
|     Carlos Lima| 7200.0|
|Carolina Freitas| 5200.0|
| Cláudio Ribeiro| 6400.0|
|  Cíntia Barbosa| 7200.0|
|  Daniel Pereira| 9100.0|
|     Diego Nunes|13200.0|
| Débora Nogueira| 6600.0|
|   Eduardo Ramos| 9800.0|
|Felipe Fernandes| 8900.0|
|  Fernanda Alves| 6500.0|
|Gabriel Ferreira|12500.0|
|   Joana Pereira| 5900.0|
|  Juliana Castro| 7500.0|
|    Juliana Melo| 8700.0|
| Larissa Barbosa| 7200.0|
+----------------+-------+
only showing top 20 rows



In [104]:
# Mostre apenas os nomes distintos dos departamentos existentes.
df.select("departamento").distinct().orderBy("departamento").show()

+------------+
|departamento|
+------------+
|  Financeiro|
|   Logística|
|   Marketing|
|   Operações|
|          RH|
|          TI|
|      Vendas|
+------------+



In [105]:
# Mostre apenas os nomes distintos dos departamentos existentes.
spark.sql("""
    select distinct(departamento)
    from funcionarios
    order by 1
""").show()

+------------+
|departamento|
+------------+
|  Financeiro|
|   Logística|
|   Marketing|
|   Operações|
|          RH|
|          TI|
|      Vendas|
+------------+



In [117]:
# Conte quantos funcionários existem na empresa.
from pyspark.sql import functions as F

df.agg(F.count("nome").alias("quantidade_funcionarios")).show()

+-----------------------+
|quantidade_funcionarios|
+-----------------------+
|                     60|
+-----------------------+



In [122]:
# Ordene os funcionários pelo salário de forma decrescente.
df.select("nome", "salario").orderBy(df.salario.desc()).show()

+----------------+-------+
|            nome|salario|
+----------------+-------+
|    André Santos|15000.0|
| Mariana Ribeiro|14800.0|
| Rodrigo Almeida|14200.0|
|     Diego Nunes|13200.0|
|Gabriel Ferreira|12500.0|
|     Mateus Lima|11800.0|
|   Roberto Souza|11200.0|
|  Patrícia Gomes|11000.0|
|   Rafael Santos|10500.0|
|   Eduardo Ramos| 9800.0|
|  Patrícia Nunes| 9700.0|
|    Tatiane Lima| 9500.0|
| Luciana Almeida| 9400.0|
|  Daniel Pereira| 9100.0|
|Felipe Fernandes| 8900.0|
|    Juliana Melo| 8700.0|
|  Pedro Henrique| 8700.0|
|  Leonardo Alves| 8700.0|
|Tatiane Ferreira| 8200.0|
|Rodrigo Ferreira| 8200.0|
+----------------+-------+
only showing top 20 rows



# Exercícios Intermediários

In [126]:
# Qual é o maior salário da empresa?
df.agg(max(df.salario)).show()

+------------+
|max(salario)|
+------------+
|     15000.0|
+------------+



In [135]:
# Qual é o salário médio de cada departamento?
df.groupBy(df.departamento).agg(F.round(F.avg(df.salario),2)).alias("salario médio").show()

+------------+----------------------+
|departamento|round(avg(salario), 2)|
+------------+----------------------+
|          TI|               8008.33|
|      Vendas|                5837.5|
|   Logística|               4114.29|
|          RH|               6071.43|
|  Financeiro|                8290.0|
|   Marketing|                5000.0|
|   Operações|               8014.29|
+------------+----------------------+



In [141]:
# Liste os 3 funcionários mais jovens.
df.select("nome", "idade").orderBy(df.idade.asc()).limit(3).show()

+----------------+-----+
|            nome|idade|
+----------------+-----+
|Gustavo Oliveira|   21|
|   Amanda Torres|   22|
|      João Pedro|   23|
+----------------+-----+



In [140]:
# Mostre o nome e o departamento dos funcionários cujo salário está entre 3000 e 7000.
df.select("nome", "departamento").filter(df.salario.between(3000,7000)).show()

+----------------+------------+
|            nome|departamento|
+----------------+------------+
|       Ana Souza|          RH|
|  Fernanda Alves|  Financeiro|
| Larissa Martins|  Financeiro|
|     Bruno Rocha|      Vendas|
|  Roberta Araújo|   Operações|
|  Felipe Cardoso|          TI|
|   Beatriz Moura|      Vendas|
|    Amanda Silva|          RH|
|     Thiago Lima|      Vendas|
|   Isabela Costa|   Marketing|
| Marcos Vinicius|   Logística|
|   Joana Pereira|  Financeiro|
|    Natália Dias|      Vendas|
|   Bruna Cardoso|   Marketing|
| Débora Nogueira|          TI|
| Cláudio Ribeiro|   Logística|
|Carolina Freitas|          RH|
|  Alexandre Lima|   Logística|
| Camila Oliveira|      Vendas|
|  Fernanda Rocha|   Marketing|
+----------------+------------+
only showing top 20 rows



In [143]:
# Qual é o total gasto em salários por departamento?
df.groupBy("departamento").agg(F.sum("salario").alias("Total Gasto")).show()

+------------+-----------+
|departamento|Total Gasto|
+------------+-----------+
|          TI|    96100.0|
|      Vendas|    46700.0|
|   Logística|    28800.0|
|          RH|    42500.0|
|  Financeiro|    82900.0|
|   Marketing|    45000.0|
|   Operações|    56100.0|
+------------+-----------+



# Exercícios Avançados

In [156]:
# Qual o departamento com o maior salário médio?
df.groupBy("departamento").agg(F.avg("salario").alias("salario_medio")).orderBy(F.col("salario_medio").desc()).limit(1).show()

+------------+-------------+
|departamento|salario_medio|
+------------+-------------+
|  Financeiro|       8290.0|
+------------+-------------+



In [157]:
# Quais funcionários têm o mesmo salário que outro colega?
# Agrupar por salário e contar
salarios_repetidos = df.groupBy("salario").count().filter("count > 1")

# Juntar para pegar os nomes
df.join(salarios_repetidos, on="salario") \
  .select("nome", "salario") \
  .show()

+----------------+-------+
|            nome|salario|
+----------------+-------+
|    Amanda Silva| 3100.0|
|  Felipe Cardoso| 3100.0|
|  Pedro Henrique| 8700.0|
|  Leonardo Alves| 8700.0|
|    Juliana Melo| 8700.0|
|Tatiane Ferreira| 8200.0|
|Rodrigo Ferreira| 8200.0|
|  Alexandre Lima| 3300.0|
|   Isabela Costa| 3300.0|
|   Bruna Cardoso| 4100.0|
| Marcos Vinicius| 4100.0|
| Larissa Barbosa| 7200.0|
|  Cíntia Barbosa| 7200.0|
|   Mariana Costa| 7200.0|
|     Carlos Lima| 7200.0|
|   Marcelo Pinto| 2800.0|
|   Ricardo Silva| 2800.0|
|     João Marcos| 2900.0|
| Gabriela Torres| 2900.0|
|  Fernanda Rocha| 3500.0|
+----------------+-------+
only showing top 20 rows



In [150]:
# Liste o funcionário mais velho de cada departamento.
df.groupBy("departamento").agg(F.max("idade").alias("idade_max")).show()

+------------+---------+
|departamento|idade_max|
+------------+---------+
|          TI|       40|
|      Vendas|       43|
|   Logística|       46|
|          RH|       47|
|  Financeiro|       50|
|   Marketing|       38|
|   Operações|       50|
+------------+---------+



In [145]:
# Mostre os funcionários cujo nome começa com a letra 'A'.
df.filter(df.nome.startswith("A")).show()

+---+--------------+------------+--------------------+-----+-------+--------------+
| id|          nome|departamento|               cargo|idade|salario|        cidade|
+---+--------------+------------+--------------------+-----+-------+--------------+
|  1|     Ana Souza|          RH|      Analista de RH|   29| 3200.0|     São Paulo|
| 14|  André Santos|  Financeiro|  Diretor Financeiro|   50|15000.0|     São Paulo|
| 21|  Amanda Silva|          RH|         Recrutadora|   27| 3100.0|Rio de Janeiro|
| 32|Anderson Gomes|   Operações|Operador de Máquinas|   40| 2700.0|        Manaus|
| 49|Alexandre Lima|   Logística| Analista de Estoque|   27| 3300.0|Belo Horizonte|
| 60| Amanda Torres|   Marketing|Assistente de Com...|   22| 2300.0|Rio de Janeiro|
+---+--------------+------------+--------------------+-----+-------+--------------+



In [144]:
# Crie uma coluna extra chamada bonus que seja 10% do salário e mostre apenas nome, salario e bonus.
df.withColumn("bonus", df.salario * 0.1).select("nome", "salario", "bonus").show()

+----------------+-------+------+
|            nome|salario| bonus|
+----------------+-------+------+
|       Ana Souza| 3200.0| 320.0|
|     Carlos Lima| 7200.0| 720.0|
|  Fernanda Alves| 6500.0| 650.0|
|   Ricardo Silva| 2800.0| 280.0|
|   Mariana Costa| 7200.0| 720.0|
|      João Pedro| 2100.0| 210.0|
|  Patrícia Gomes|11000.0|1100.0|
|Gabriel Ferreira|12500.0|1250.0|
| Larissa Martins| 3900.0| 390.0|
|     Bruno Rocha| 5400.0| 540.0|
|   Camila Torres| 2500.0| 250.0|
|   Eduardo Ramos| 9800.0| 980.0|
|    Juliana Melo| 8700.0| 870.0|
|    André Santos|15000.0|1500.0|
|  Roberta Araújo| 4300.0| 430.0|
|  Felipe Cardoso| 3100.0| 310.0|
|   Beatriz Moura| 6100.0| 610.0|
|   Marcelo Pinto| 2800.0| 280.0|
|    Tatiane Lima| 9500.0| 950.0|
|     Diego Nunes|13200.0|1320.0|
+----------------+-------+------+
only showing top 20 rows

